In [1]:
#pip install wget

In [2]:
#pip install prettytable

In [3]:
# usual imports
import os
import itertools
from collections import OrderedDict, defaultdict

# data sciency 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

# web scraping tools
from bs4 import BeautifulSoup
import wget

# plot formatting tools
from prettytable import PrettyTable
from IPython.core.display import HTML
from pprint import pprint

In [4]:
# basic functions for deafultdict
def constant_factory(value):
    '''Always produces a constant value; used for defaultdict'''
    return next(itertools.repeat(value))

def caption(msg, tablenum):
    '''helps convert text into a table caption'''
    return "<br><br>Table %d - %s</b>" % (tablenum, msg)

### Initial inflation multipliers to get 2019 dollars from 2018

In [5]:
inflation_multiplier = {
    2018: 1.02
}

### Regions used 

In [6]:
Region_East = 'east'
Region_Midwest = 'midwest'
Region_South = 'south'
Region_West = 'west'
Region_Base = 'base' #used for states without a region (ex. alaska/hawaii)

#map state to a region in order to adjust for weights

state_to_region_map = (
    {
    'PA': Region_East, 'NJ': Region_East, 'NY': Region_East, 'CT': Region_East, 'MA': Region_East,
    'NH': Region_East, 'VT': Region_East, 'ME': Region_East, 'RI': Region_East,
    'OH': Region_Midwest, 'IL': Region_Midwest, 'IN': Region_Midwest, 'WI': Region_Midwest, 'MI': Region_Midwest,
    'MN': Region_Midwest, 'IA': Region_Midwest, 'MO': Region_Midwest, 'KS': Region_Midwest, 'NE': Region_Midwest,
    'SD': Region_Midwest, 'ND': Region_Midwest,
    'TX': Region_South, 'OK': Region_South, 'AR': Region_South, 'LA': Region_South, 'MS': Region_South,
    'AL': Region_South, 'GA': Region_South, 'FL': Region_South, 'SC': Region_South, 'NC': Region_South,
    'VA': Region_South, 'WV': Region_South, 'KY': Region_South, 'TN': Region_South, 'MD': Region_South,
    'DE': Region_South, 'DC': Region_South,
    'CA': Region_West, 'OR': Region_West, 'WA': Region_West, 'NV': Region_West, 'ID': Region_West,
    'UT': Region_West, 'AZ': Region_West, 'MT': Region_West, 'WY': Region_West, 'CO': Region_West,
    'NM': Region_West, 
    'AK': Region_Base, 'HI': Region_Base, 'AS': Region_Base, 'GU': Region_Base, 'MP': Region_Base, 'PR': Region_Base,
    'VI': Region_Base    
})

In [7]:
# Create inverse mapping of region to list of states
region_to_state_mapping = { }
for state, region in state_to_region_map.items():
    if region in region_to_state_mapping:
        region_to_state_mapping[region].append(state)
    else:
        region_to_state_mapping[region] = [state]

#### Inverse mapping from region to states

In [8]:
state_to_statename_map = {'Federal': 'Federal', 'AS':'American Samoa','AK':'Alaska', 'AL':'Alabama', 'AR':'Arkansas', 'AZ':'Arizona',
                         'CA':'California', 'CO':'Colorado', 'CT':'Connecticut', 'DC':'District of Columbia', 
                         'DE':'Delaware', 'FL':'Florida', 'GA':'Georgia', 'GU':'Guam', 'HI':'Hawaii', 'IA':'Iowa', 
                         'ID':'Idaho', 'MA':'Massachusetts', 'MD':'Maryland', 'ME':'Maine', 'MI':'Michigan',
                         'MN':'Minnesota', 'MO':'Missouri', 'MP':'Mariana Islands', 'MS':'Mississippi', 'MT':'Montana', 'NC':'North Carolina',
                         'ND':'North Dakota', 'NE':'Nebraska', 'NH':'New Hampshire', 'NJ':'New Jersey', 'NM':'New Mexico',
                         'NV':'Nevada', 'NY':'New York', 'OH':'Ohio', 'OK':'Oklahoma', 'OR':'Oregon', 'PA':'Pennsylvania', 'PR':'Puerto Rico',
                         'RI':'Rhode Island', 'SC':'South Carolina', 'SD':'South Dakota', 'TN':'Tennessee', 'TX':'Texas',
                         'UT':'Utah', 'VA':'Virgina', 'VI':'Virgin Islands', 'VT':'Vermont', 'WA':'Washington','WI':'Wisconsin', 'WV':'West Virginia',
                         'WY':'Wyoming'}

statename_to_state_map = {statename:state for (state, statename) in state_to_statename_map.items()}

#### Regional differences for food data

In [9]:
food_regional_multipliers = {
    Region_East: 0.08,
    Region_West: 0.11,
    Region_South: -0.07,
    Region_Midwest: -0.05
}

#### Define the model year

In [10]:
model_year = 2019

## Data Sources

### Consumer Expenditure Report from BLS

In [11]:
# Get CEX for 2018
# wget.download('http://www.bls.gov/cex/2019/combined/cusize.xlsx')

### Free Market Rent Data from HUD

In [12]:
# wget.download('https://www.huduser.gov/portal/datasets/fmr/fmr2019/FY2019_4050_FMRs_rev2.xlsx')

In [13]:
# create a dictionary for counties - future use
counties = { }

### Medical Expenditure from AHRQ

In [14]:
# load insurance data and parse through html
# wget.download('https://www.meps.ahrq.gov/data_stats/summ_tables/insr/state/series_10/2018/txe1.htm')

### Tax data

In [15]:
# Federal taxes are taken from the Urban-Brooking Tax Policy Center Microsimulation Model.
# It includes: Individual incomes taxes, payroll taxes, corporate income tax, estate tax, and excise tax.

# wget.download('https://www.taxpolicycenter.org/file/157131/download?token=_7XWdK7j')

In [16]:
# State tax rate is taken from the state income tax rate, as reported by the CCH State Tax Handbook 2015
# The URL provided in the technical documentation did not produce the data required
# We will use state income tax data from taxfoundation.org and not the CCH State Tax Handbook 2015

# wget.download('https://files.taxfoundation.org/20201130110549/State-Individual-Income-Tax-Rates-and-Brackets-for-2020-U.xlsx')

### Housing Costs 
#### For this project we are assuming a three-person household of two adults (19-50yrs) and one young child (4 yrs). Thus, a two-bedroom apartment is the default assumption for rent. 
#### The counties are identified by the FIPS code (state code+county code+subcounty code)

In [17]:
# Fair Market Rent (FMR) Data
fmr_data = { }

def pad_county(county):
    '''Pad counties to three digits when we need to construct one manually'''
    return '%03d' % county

def pad_fips(fip):
    '''add 99999 to end of fip code (which nullifies the subcounty identifier)'''
    return int(str(fip) + '99999')

In [18]:
fmr_data2019 = pd.read_excel("/Users/cj/Documents/FY2019_4050_FMRs_rev2.xlsx")
fmr_data2019 = fmr_data2019[["fmr_1", "county", "cousub", "countyname", "fips2010", "pop2010", "state", "state_alpha"]]
fmr_data2019.rename(columns={'fips2010':'fips', 'pop2010':'pop'}, inplace=True)
fmr_data2019['fips'] = fmr_data2019['fips'].map(lambda x: (x / 100000) * 100000 + 99999)
fmr_data2019 = fmr_data2019.drop_duplicates(subset=["state", "countyname"])

In [19]:
# add region column, the defaultdict will use region base if the state is not in the initial state to region mapping
fmr_data2019['region'] = fmr_data2019['state_alpha'].map(lambda x: state_to_region_map[x])

In [20]:
# counties employed
employed_counties = [( )]

employed_fips = []
for (county, state), in employed_counties:
    selector = fmr_data2019[fmr_data2019]['state_alpha'] == ['state']['countyname'].map(str.lower) == county.lower()
    if not np.any(selector):
        print("missed %s %s" % (county, state))
    else:
        employed_fips.append(fmr_data2019[fmr_data2019]['state_alpha']==['state'][selector]['fips'].values[0])

<>:6: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:6: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-20-476af8df7278>:6: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  selector = fmr_data2019[fmr_data2019]['state_alpha'] == ['state']['countyname'].map(str.lower) == county.lower()


ValueError: not enough values to unpack (expected 1, got 0)

In [ ]:
print(len(employed_counties))
print(len(employed_fips))

### Food Costs - USDA's low-cost food plan national average in June 2019
#### Adult food consumption costs are estimated by averaging the low-cost plan food costs for males and females between 19-50
#### Child food costs are estimated using the various categories in the low-cost food plan based on the child age assumptions (4 year old - 'young child'

In [25]:
national_monthly_food_cost = {
    2019: {'base': np.average([242.80,205.00]) + 137.40}
}

In [26]:
# regional adjustment
for region in food_regional_multipliers:
    national_monthly_food_cost[2019][region] = \
        national_monthly_food_cost[2019]['base'] * (1 + food_regional_multipliers[region])

national_monthly_food_cost_df = pd.DataFrame.from_dict(national_monthly_food_cost)

### Child Care Costs - state level estimates published by the National Association of Child Care Resource and Referral Agencies in 2016
### Values were inflated from 2016 to 2019 dollars using the Consumer Price Index inflation multiplier from the BLS